In [1]:
import datetime
from dateutil.parser import parse
import itertools
from itertools import product
import math
import matplotlib.pyplot as plt
from multiprocessing.dummy import Pool as ThreadPool
import nolds
import numpy as np
import os
import pandas as pd
import pathlib
from PreprocessFcns import *
import pywt
import random
import scipy
from scipy.signal import butter, welch, filtfilt, resample
from scipy.stats import skew, kurtosis, entropy, pearsonr
import seaborn as sns
import sklearn
from sklearn import multiclass
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import LeaveOneGroupOut
import time
%matplotlib inline

In [5]:
# set path to folder containing clinic watch data
path = r'//FS2.smpp.local\\RTO\\CIS-PD Study\MJFF Curation\Finalized Dataset'
# set path to destination folder
dest = r'//FS2.smpp.local\\RTO\\CIS-PD Study\Watch Features Data'
#---------------------------------------------------------------------------------------------------------

In [3]:
VisitNumber = {
    '2 Weeks: Time 0'   : 0,
    '2 Weeks: Time 30'  : 1,
    '2 Weeks: Time 60'  : 2,
    '2 Weeks: Time 90'  : 3,
    '2 Weeks: Time 120' : 4,
    '2 Weeks: Time 150' : 5,
    '1 Month'           : 6
}

ClinicTasks = {
    'Stndg'    : 'Standing',
    'Wlkg'     : 'Walking',
    'WlkgCnt'  : 'Walking while counting',
    'FtnR'     : 'Finger to nose--right hand',
    'FtnL'     : 'Finger to nose--left hand',
    'RamR'     : 'Alternating right hand movements',
    'RamL'     : 'Alternating left hand movements',
    'SitStand' : 'Sit to stand',
    'Drwg'     : 'Drawing on a paper',
    'Typg'     : 'Typing on a computer keyboard',
    'NtsBts'   : 'Assembling nuts and bolts',
    'Drnkg'    : 'Taking a glass of water and drinking',
    'Sheets'   : 'Organizing sheets in a folder',
    'Fldg'     : 'Folding towels',
    'Sitng'    : 'Sitting'
}

In [34]:
ClipData = pd.read_csv(os.path.join(dest, 'Clinic Data', 'ClinicDataFeaturesX.csv'))[['SubjID', 'Visit', 'TaskAbb', 'Clip', 
                                                                                      'Tremor', 'Bradykinesia', 'Dyskinesia', 
                                                                                      'Overall']]

ClipFreqData = pd.DataFrame()
for clip in ClipData.iterrows():
    clipacc = pd.read_csv(os.path.join(dest, 'Clinic Data', 'Feature Clips', 
                                       str(int(clip[1]['SubjID'])) + '_' + str(clip[1]['Visit']) + '_' + 
                                       clip[1]['TaskAbb'] + '_' + str(clip[1]['Clip']) + '.csv'), 
                          parse_dates = ['Timestamp'])
    ClipFreqData['SubjID'] = [int(clip[1]['SubjID'])]
    ClipFreqData['Visit'] = [clip[1]['Visit']]
    ClipFreqData['TaskAbb'] = [clip[1]['TaskAbb']]
    ClipFreqData['Clip'] = [clip[1]['Clip']]
    ClipFreqData['Tremor'] = [clip[1]['Tremor']]
    ClipFreqData['Bradykinesia'] = [clip[1]['Bradykinesia']]
    ClipFreqData['Dyskinesia'] = [clip[1]['Dyskinesia']]
    ClipFreqData['Overall'] = [clip[1]['Overall']]
    ClipFreqData['0-2'] = [np.mean(power_spectra_welch(clipacc, 0, 2)['PSD_magnitude'].values)]
    ClipFreqData['2-4'] = [np.mean(power_spectra_welch(clipacc, 2, 4)['PSD_magnitude'].values)]
    ClipFreqData['4-6'] = [np.mean(power_spectra_welch(clipacc, 4, 6)['PSD_magnitude'].values)]
    ClipFreqData['6-8'] = [np.mean(power_spectra_welch(clipacc, 6, 8)['PSD_magnitude'].values)]
    ClipFreqData['8-10'] = [np.mean(power_spectra_welch(clipacc, 8, 10)['PSD_magnitude'].values)]
    ClipFreqData['10-12'] = [np.mean(power_spectra_welch(clipacc, 10, 12)['PSD_magnitude'].values)]
    ClipFreqData['12-14'] = [np.mean(power_spectra_welch(clipacc, 12, 14)['PSD_magnitude'].values)]
    ClipFreqData['14-16'] = [np.mean(power_spectra_welch(clipacc, 14, 16)['PSD_magnitude'].values)]
    ClipFreqData['16-18'] = [np.mean(power_spectra_welch(clipacc, 16, 18)['PSD_magnitude'].values)]
    ClipFreqData['18-20'] = [np.mean(power_spectra_welch(clipacc, 18, 20)['PSD_magnitude'].values)]
    print(ClipFreqData)
    break

   SubjID  Visit TaskAbb  Clip  Tremor  Bradykinesia  Dyskinesia  Overall  \
0    1003      0   Drnkg     0     1.0           1.0         0.0      1.0   

        0-2  2-4           4-6  6-8          8-10  10-12     12-14  14-16  \
0  0.000002  NaN  4.340081e-07  NaN  8.791187e-07    NaN  0.000001    NaN   

     16-18  18-20  
0  0.00002    NaN  


In [32]:
np.mean(power_spectra_welch(pd.read_csv(os.path.join(dest, 'Clinic Data', 'Feature Clips', '1003_0_Drnkg_0.csv')), 2, 4)['PSD_magnitude'].values)

C:\Users\rpieper\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\rpieper\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan